In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add scripts folder to path
sys.path.append('scripts')
from db_to_pandas import DatabaseConverter

# Configure display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully")

# Create a database converter instance
converter = DatabaseConverter()


# Arena Analysis - Biggest Arenas

Analysis of arena capacities and configurations from the arena_snapshots table.

In [ ]:
# Load arena snapshots data
arenas_df = converter.get_arena_snapshots_df()
print(f"Loaded {len(arenas_df)} arena snapshots")
print(f"Date range: {arenas_df['created_at'].min()} to {arenas_df['created_at'].max()}")

# Display basic information about the arenas dataset
print(f"\nDataset Info:")
print(f"Shape: {arenas_df.shape}")
print(f"\nColumns: {list(arenas_df.columns)}")
print(f"\nData types:")
print(arenas_df.dtypes)

In [ ]:
# Calculate total capacity for each arena
arenas_df['total_capacity'] = (arenas_df['bleachers_capacity'] + 
                               arenas_df['lower_tier_capacity'] + 
                               arenas_df['courtside_capacity'] + 
                               arenas_df['luxury_boxes_capacity'])

# Display sample of arena data
print("Arena Snapshots Sample:")
display_cols = ['team_id', 'created_at', 'bleachers_capacity', 'lower_tier_capacity', 
                'courtside_capacity', 'luxury_boxes_capacity', 'total_capacity']
arenas_df[display_cols].head(10)

In [ ]:
# Arena Capacity Statistics
print("Arena Capacity Statistics:")
capacity_cols = ['bleachers_capacity', 'lower_tier_capacity', 'courtside_capacity', 
                'luxury_boxes_capacity', 'total_capacity']
print(arenas_df[capacity_cols].describe())

print(f"\nTotal unique teams with arena data: {arenas_df['team_id'].nunique()}")
print(f"Average snapshots per team: {len(arenas_df) / arenas_df['team_id'].nunique():.2f}")

In [ ]:
# Find the biggest arenas by total capacity
print("TOP 20 BIGGEST ARENAS (by total capacity):")
print("="*60)

# Get latest snapshot for each team to avoid duplicates
latest_arenas = arenas_df.loc[arenas_df.groupby('team_id')['created_at'].idxmax()]

# Sort by total capacity
biggest_arenas = latest_arenas.sort_values('total_capacity', ascending=False).head(20)

for idx, row in biggest_arenas.iterrows():
    print(f"Team {row['team_id']}: {row['total_capacity']:5d} total "
          f"(B:{row['bleachers_capacity']:5d}, L:{row['lower_tier_capacity']:4d}, "
          f"C:{row['courtside_capacity']:3d}, LB:{row['luxury_boxes_capacity']:2d})")
    
print(f"\nLargest arena capacity: {biggest_arenas.iloc[0]['total_capacity']:,}")
print(f"Smallest in top 20: {biggest_arenas.iloc[19]['total_capacity']:,}")

In [ ]:
# Analyze arena configurations and patterns
print("ARENA CONFIGURATION ANALYSIS:")
print("="*50)

# Calculate seat type percentages for latest arenas
latest_arenas['bleachers_pct'] = (latest_arenas['bleachers_capacity'] / latest_arenas['total_capacity'] * 100)
latest_arenas['lower_tier_pct'] = (latest_arenas['lower_tier_capacity'] / latest_arenas['total_capacity'] * 100)
latest_arenas['courtside_pct'] = (latest_arenas['courtside_capacity'] / latest_arenas['total_capacity'] * 100)
latest_arenas['luxury_boxes_pct'] = (latest_arenas['luxury_boxes_capacity'] / latest_arenas['total_capacity'] * 100)

print("Average seat distribution:")
print(f"Bleachers: {latest_arenas['bleachers_pct'].mean():.1f}%")
print(f"Lower Tier: {latest_arenas['lower_tier_pct'].mean():.1f}%")
print(f"Courtside: {latest_arenas['courtside_pct'].mean():.1f}%")
print(f"Luxury Boxes: {latest_arenas['luxury_boxes_pct'].mean():.1f}%")

# Find most common configurations
print(f"\nMost common total capacities:")
capacity_counts = latest_arenas['total_capacity'].value_counts().head(10)
for capacity, count in capacity_counts.items():
    print(f"{capacity:5d} seats: {count:3d} teams ({count/len(latest_arenas)*100:.1f}%)")

In [ ]:
# Visualize arena capacity distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Total capacity distribution
axes[0,0].hist(latest_arenas['total_capacity'], bins=30, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Total Arena Capacity Distribution')
axes[0,0].set_xlabel('Total Capacity')
axes[0,0].set_ylabel('Number of Arenas')

# Seat type distributions
seat_types = ['bleachers_capacity', 'lower_tier_capacity', 'courtside_capacity', 'luxury_boxes_capacity']
colors = ['skyblue', 'lightgreen', 'orange', 'lightcoral']

for i, (seat_type, color) in enumerate(zip(seat_types, colors)):
    if i == 0:
        axes[0,1].hist(latest_arenas[seat_type], bins=20, alpha=0.7, color=color, 
                       label=seat_type.replace('_capacity', ''), edgecolor='black')
    else:
        axes[0,1].hist(latest_arenas[seat_type], bins=20, alpha=0.7, color=color, 
                       label=seat_type.replace('_capacity', ''), edgecolor='black')

axes[0,1].set_title('Seat Type Capacity Distributions')
axes[0,1].set_xlabel('Capacity')
axes[0,1].set_ylabel('Number of Arenas')
axes[0,1].legend()

# Box plot of capacities by seat type
capacity_data = [latest_arenas['bleachers_capacity'], 
                latest_arenas['lower_tier_capacity'],
                latest_arenas['courtside_capacity'], 
                latest_arenas['luxury_boxes_capacity']]
axes[1,0].boxplot(capacity_data, labels=['Bleachers', 'Lower Tier', 'Courtside', 'Luxury Boxes'])
axes[1,0].set_title('Capacity Distribution by Seat Type')
axes[1,0].set_ylabel('Capacity')
axes[1,0].tick_params(axis='x', rotation=45)

# Seat type percentage pie chart
avg_percentages = [latest_arenas['bleachers_pct'].mean(),
                  latest_arenas['lower_tier_pct'].mean(),
                  latest_arenas['courtside_pct'].mean(),
                  latest_arenas['luxury_boxes_pct'].mean()]

axes[1,1].pie(avg_percentages, labels=['Bleachers', 'Lower Tier', 'Courtside', 'Luxury Boxes'],
              autopct='%1.1f%%', startangle=90, colors=colors)
axes[1,1].set_title('Average Seat Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Analyze arena evolution over time (for teams with multiple snapshots)
teams_with_multiple_snapshots = arenas_df.groupby('team_id').size()
teams_with_changes = teams_with_multiple_snapshots[teams_with_multiple_snapshots > 1]

print(f"ARENA EVOLUTION ANALYSIS:")
print(f"="*40)
print(f"Teams with multiple arena snapshots: {len(teams_with_changes)}")
print(f"Teams with single snapshot: {len(teams_with_multiple_snapshots) - len(teams_with_changes)}")

if len(teams_with_changes) > 0:
    print(f"\nTop 10 teams with most arena changes:")
    top_changing_teams = teams_with_changes.sort_values(ascending=False).head(10)
    for team_id, count in top_changing_teams.items():
        print(f"Team {team_id}: {count} snapshots")
        
    # Show evolution for top changing team
    most_active_team = top_changing_teams.index[0]
    team_evolution = arenas_df[arenas_df['team_id'] == most_active_team].sort_values('created_at')
    
    print(f"\nCapacity evolution for Team {most_active_team}:")
    for _, row in team_evolution.iterrows():
        print(f"{row['created_at']}: {row['total_capacity']} total "
              f"(B:{row['bleachers_capacity']}, L:{row['lower_tier_capacity']}, "
              f"C:{row['courtside_capacity']}, LB:{row['luxury_boxes_capacity']})")
else:
    print("No teams with multiple snapshots found.")

# Arena Demand Statistical Analysis

Statistical analysis of arena demand given prices and other attributes.

In [ ]:
# Initialize database converter
converter = DatabaseConverter()

# Get all games data
all_games = converter.get_games_df(limit=100000)
print(f"Loaded {len(all_games)} games")
print(f"Date range: {all_games['date'].min()} to {all_games['date'].max()}")

# keep only the latest 1000 games
N = 1000
games_df = all_games.iloc[-N:]
print(f"Filtered to latest {len(games_df)} games")

In [ ]:
# Display basic information about the games dataset
print("Dataset Info:")
print(f"Shape: {games_df.shape}")
print(f"\nColumns: {list(games_df.columns)}")
print(f"\nData types:")
print(games_df.dtypes)

In [ ]:
# Display sample of latest games data
print("Latest Games Sample:")
display_cols = ['game_id', 'home_team_id', 'date', 'game_type', 'calculated_revenue', 'bleachers_attendance', 'lower_tier_attendance', 'courtside_attendance', 'luxury_boxes_attendance',
                'bleachers_price', 'lower_tier_price', 'courtside_price', 'luxury_boxes_price']
games_df[display_cols].head(10)

In [ ]:
# find row with bleachers attendance of bleacher_test
bleacher_test = 14590

attendance_row = all_games[all_games['bleachers_attendance'] == bleacher_test]
if not attendance_row.empty:
    print(f"\nRow with bleachers attendance of {bleacher_test} found:")
    print(attendance_row[display_cols])
else:
    print(f"\nNo row with bleachers attendance of {bleacher_test} found.")
    closest_row = all_games.iloc[(all_games['bleachers_attendance'] - bleacher_test).abs().argsort()[:1]]
    print(f"Closest match:")
    print(closest_row)
    

In [ ]:
# find row with lower_tier attendance of lower_tier_test
lower_tier_test = 3122
attendance_row = all_games[all_games['lower_tier_attendance'] == lower_tier_test]
if not attendance_row.empty:
    print(f"\nRow with lower_tier attendance of {lower_tier_test} found:")
    print(attendance_row[display_cols])
else:
    print(f"\nNo row with lower_tier attendance of {lower_tier_test} found.")
    closest_row = all_games.iloc[(all_games['lower_tier_attendance'] - lower_tier_test).abs().argsort()[:1]]
    print(f"Closest match:")
    print(closest_row)


In [ ]:
valid_games = all_games[
    (all_games['bleachers_price'] > 0) &
    (all_games['lower_tier_price'] > 0) &
    (all_games['courtside_price'] > 0) &
    (all_games['luxury_boxes_price'] > 0) &
    (all_games['bleachers_attendance'] >= 0) &
    (all_games['lower_tier_attendance'] >= 0) &
    (all_games['courtside_attendance'] >= 0) &
    (all_games['luxury_boxes_attendance'] >= 0)
]
print(f"\nFiltered valid games: {len(valid_games)}")
# Display sample of latest games data
print("Latest Games Sample:")
valid_games[display_cols].sort_values(by='calculated_revenue', ascending=False).head(10)

In [ ]:
# Basic statistics
print("Basic Statistics:")
print("\n1. Attendance Statistics:")
attendance_cols = ['bleachers_attendance', 'lower_tier_attendance', 'courtside_attendance', 'luxury_boxes_attendance', 'total_attendance']
print(valid_games[attendance_cols].describe())

print("\n2. Pricing Statistics:")
price_cols = ['bleachers_price', 'lower_tier_price', 'courtside_price', 'luxury_boxes_price']
print(valid_games[price_cols].describe())

print("\n3. Revenue Statistics:")
revenue_cols = ['ticket_revenue', 'calculated_revenue']
print(valid_games[revenue_cols].describe())

In [ ]:
# Game type distribution
print("Game Type Distribution:")
game_type_counts = valid_games['game_type'].value_counts()
print(game_type_counts)
print(f"\nGame type percentages:")
print((game_type_counts / len(valid_games) * 100).round(2))

In [ ]:
# Get team IDs for season 68 and league_id 1
team_ids = converter.get_team_ids_for_league(season=68, league_id=1)
print(f"Found {len(team_ids)} teams in season 68, league 1:")
print(team_ids)

In [ ]:
# Test the new functions for standings calculation
season = 68
league_id = 1

# Get regular season games
print("Getting regular season games...")
reg_season_games = converter.get_league_regular_season_games(season, league_id)
print(f"Found {len(reg_season_games)} regular season games")

# Group into rounds
print("\nGrouping games into rounds...")
rounds = converter.group_games_into_rounds(reg_season_games)
print(f"Found {len(rounds)} rounds")

# Display round summary
print("\nRound Summary:")
for round_info in rounds[:5]:  # Show first 5 rounds
    print(f"Round {round_info['round_number']}: {round_info['game_count']} games, "
          f"median time: {round_info['median_time'].strftime('%Y-%m-%d %H:%M')}, "
          f"max time diff: {round_info['max_time_diff_minutes']:.1f} min, "
          f"valid: {round_info['valid_round']}")

# Calculate standings before round 5
print(f"\nStandings before Round 5:")
standings_r5 = converter.calculate_standings_before_round(season, league_id, 5)


# Get wins for a specific team before round 5
if not standings_r5.empty:
    test_team_id = standings_r5.iloc[0]['team_id']
    wins = converter.get_team_wins_before_round(test_team_id, season, league_id, 5)
    print(f"\nTeam {test_team_id} had {wins} wins before Round 5")

standings_r5

In [ ]:
# Test regular season games function
season = 69
league_id = 1
games = converter.get_league_regular_season_games(season, league_id)
print(f"Found {len(games)} regular season games for season {season}, league {league_id}")

In [ ]:
# Test grouping into rounds
rounds = converter.group_games_into_rounds(games)
print(f"Grouped into {len(rounds)} rounds")
print(f"First round has {rounds[0]['game_count']} games" if rounds else "No rounds found")

In [ ]:
# Test standings calculation
round_number = 4
standings = converter.calculate_standings_before_round(season, league_id, round_number)
print(f"Standings before round {round_number} | season {season}, league {league_id}:")
standings.head()

In [ ]:
# Debug: Check actual game scores from first 2 rounds
print("Sample games from first 2 rounds:")
if len(rounds) >= 2:
    for round_num in [0, 1]:  # First two rounds
        round_games = rounds[round_num]['games']
        print(f"\nRound {round_num + 1} games:")
        for _, game in round_games.iterrows():
            print(f"  Game {game['game_id']}: Team {game['home_team_id']} vs {game['away_team_id']} - Scores: {game['score_home']} vs {game['score_away']}")
            break  # Just show first game of each round

In [ ]:
# Check if any games have actual scores
games_with_scores = games[(games['score_home'].notna()) & (games['score_away'].notna())]
print(f"Games with scores: {len(games_with_scores)} out of {len(games)}")

if len(games_with_scores) > 0:
    print("Sample game with scores:")
    sample = games_with_scores.iloc[0]
    print(f"Game {sample['game_id']}: {sample['home_team_id']} vs {sample['away_team_id']} - {sample['score_home']} vs {sample['score_away']}")
else:
    print("No games have scores recorded yet")

In [ ]:
# Test team wins function
if not standings.empty:
    test_team = int(standings.iloc[0]['team_id'])
    wins = converter.get_team_wins_before_round(test_team, season, league_id, 3)
    print(f"Team {test_team} has {wins} wins before round 3")

In [ ]:
def get_current_season() -> int:
    """
    Get the current season from the database.
    
    Returns:
        int: The current season number.
    """
    with converter.get_connection() as conn:
        query = "SELECT MAX(season) as current_season FROM games"
        result = pd.read_sql_query(query, conn)
        # convert to int if not empty

        if not result.empty:
            return int(result.iloc[0]['current_season'])
        else:
            raise ValueError("No seasons found in database.")
    
print(get_current_season())

In [ ]:
from typing import Tuple
def get_league_info_of_team(team_id, season: int = None) -> Tuple[int, int, int]:
    """
    Get the league ID for a given team.
    If current season is not specified, use the latest season.
    For the current season use team_info table.
    For previous seasons use team_league_history table.
    
    Args:
        team_id (int): The team ID to look up.
        season (int, optional): The season to check. Defaults to None (current season).

    Returns:
        Tuple[int, int, int]: A tuple containing league_id, league_level, and country_id.
        
    """
    current_season = get_current_season()
    if season is None:
        season = get_current_season()
    
    with converter.get_connection() as conn:
        if season == current_season:
            query = "SELECT league_id, league_level, country_id FROM team_info WHERE bb_team_id = ?"
            result = pd.read_sql_query(query, conn, params=[team_id])
        else:
            query = "SELECT league_id, league_level FROM team_league_history WHERE team_id = ? AND season = ?"
            result_1 = pd.read_sql_query(query, conn, params=[team_id, season])
            query = "SELECT country_id FROM team_info WHERE bb_team_id = ?"
            result_2 = pd.read_sql_query(query, conn, params=[team_id])
            if not result_1.empty and not result_2.empty:
                result = pd.merge(result_1, result_2, left_index=True, right_index=True)
            else:
                raise ValueError(f"No league found for team {team_id} in season {season}")

        if not result.empty:
            return (int(result.iloc[0]['league_id']), int(result.iloc[0]['league_level']), int(result.iloc[0]['country_id']))
        else:
            raise ValueError(f"No league found for team {team_id} in season {season}")
test_league = get_league_info_of_team(29613)
test_league_id = test_league[0]
print(get_league_info_of_team(29613, 68))
print(get_league_info_of_team(29613))  # Should use current season
print(get_league_info_of_team(29613, 64))  # Should use current season

In [ ]:
games_df = converter.get_league_regular_season_games(season=69, league_id=test_league_id)

In [ ]:
round_list =converter.group_games_into_rounds(games_df)
print(round_list)

In [ ]:
game_id = 135202277
round = None
for r in round_list:
    if str(game_id) in r['games']['game_id'].values:
        round = r['round_number']
        break
print(round)

In [ ]:
def get_league_info_of_game(game_id: int) -> Tuple[int, int, int, int]:
    """ Get the league information of a game by:
    - finding the season of the game (either season column or via the startdate)
    - finding the league of the game from the home_team_id
    Args:
        game_id (int): The game ID to look up.
    Returns:
        Tuple[int, int, int]: A tuple containing the league ID, season, league level, and league country ID.
    """

    # Get the game details
    with converter.get_connection() as conn:
        query = "SELECT season, home_team_id, game_type FROM games WHERE game_id = ?"
        game_info = pd.read_sql_query(query, conn, params=[game_id])
    
    if game_info.empty:
        raise ValueError(f"Game {game_id} not found in database.")
    
    # Skip game whose game_type is not 'league.rs' or 'league.rs.tv'
    if game_info.iloc[0]['game_type'] not in ['league.rs', 'league.rs.tv']:
        raise ValueError(f"Game {game_id} is not a regular season game.")
    
    season = int(game_info.iloc[0]['season'])
    home_team_id = int(game_info.iloc[0]['home_team_id'])
    # Get the league ID for the home team
    league_id, league_level, league_country = get_league_info_of_team(home_team_id, season)

    return league_id, season, league_level, league_country

def league_round_info_of_game(game_id: int, league_id: int = None, season: int = None) -> int:
    """
    Get the round number of a game by:
    - finding the season of the game (either season column or via the startdate)
    - finding the league of the game from the home_team_id
    - using get_league_regular_season_games
    - grouping the games into rounds
    - iterating through the rounds to find the game_id
    Args:
        game_id (int): The game ID to look up.
    Returns:
        int: The round number of the game.
    """
    if league_id is None or season is None:
        # Get league info of the game
        league_id, season, _, _ = get_league_info_of_game(game_id)

    # Get regular season games for that league and season
    games_df = converter.get_league_regular_season_games(season, league_id)

    # Group into rounds
    rounds = converter.group_games_into_rounds(games_df)
    # Skip if any rounds are invalid round_info['valid_round']
    
    # Find the round containing the game
    for round_info in rounds:
        if str(game_id) in round_info['games']['game_id'].values:
            return round_info['round_number']
    
    raise ValueError(f"Game {game_id} not found in any round.")
print(league_round_info_of_game(135202291))  # Should return the round number for

In [ ]:
# get rows in valid_games without season information
no_season_games = valid_games[valid_games['season'].isna()]
print(f"Games without season information: {len(no_season_games)}")
assert len(no_season_games) == 0, "There should be no games without season information"
# get rows in valid_games where game_type is 'league.rs' or 'league.rs.tv'
league_rs_games = valid_games[valid_games['game_type'].isin(['league.rs', 'league.rs.tv'])]
print(f"League regular season games: {len(league_rs_games)}")

# get rows in league_rs_games where league_round_of_game returns a positive integer
valid_league_games = []
for i, (_, row) in enumerate(league_rs_games.iterrows()):
    if i % 100 == 0:
        print(f"Processing game {i + 1} of {len(league_rs_games)}...", end='\r')
    league_id, season, league_level, country_id = get_league_info_of_game(row['game_id'])

    if league_level > 3:
        continue

    round_number = league_round_info_of_game(row['game_id'], league_id=league_id, season=season)
    if round_number is None or round_number <= 0:
        continue
    row['round_number'] = round_number
    row['league_id'] = league_id
    row['league_level'] = league_level
    row['country_id'] = country_id
    row['is_tv_game'] = row['game_type'] == 'league.rs.tv'
    # get standings before round
    df_standings = converter.calculate_standings_before_round(season=season, league_id=league_id, round_number=round_number)

    # get row where team_id is the home_team_id and store `wins`, `losses`, `points_diff`
    home_team_id = row['home_team_id']
    if home_team_id in df_standings['team_id'].values:
        standings_row = df_standings[df_standings['team_id'] == home_team_id]
        row['home_wins'] = standings_row['wins'].values[0]
        row['home_losses'] = standings_row['losses'].values[0]
        row['home_points_diff'] = standings_row['point_diff'].values[0]
    else:
        raise ValueError(f"Home team {home_team_id} not found in standings.")
    away_team_id = row['away_team_id']
    if away_team_id in df_standings['team_id'].values:
        standings_row = df_standings[df_standings['team_id'] == away_team_id]
        row['away_wins'] = standings_row['wins'].values[0]
        row['away_losses'] = standings_row['losses'].values[0]
        row['away_points_diff'] = standings_row['point_diff'].values[0]
    else:
        raise ValueError(f"Away team {away_team_id} not found in standings.")
    
    # get previous game performance `home_won_prev_game`
    if round_number > 1:
        prev_standings = converter.calculate_standings_before_round(season=season, league_id=league_id, round_number=round_number - 1)
        if home_team_id in prev_standings['team_id'].values:
            prev_row = prev_standings[prev_standings['team_id'] == home_team_id]
            # check if new standings row has more wins than previous for home team
            row['home_won_prev_game'] = prev_row['wins'].values[0] < row['home_wins']
        else:
            raise ValueError(f"Home team {home_team_id} not found in previous standings.")
        if away_team_id in prev_standings['team_id'].values:
            prev_row = prev_standings[prev_standings['team_id'] == away_team_id]
            # check if new standings row has more wins than previous for away team
            row['away_won_prev_game'] = prev_row['wins'].values[0] < row['away_wins']
        else:
            raise ValueError(f"Away team {away_team_id} not found in previous standings.")
    else:
        row['home_won_prev_game'] = None

    # get this season's league level vs previous season's league level for both teams
    assert season > 1, "Season must be greater than 1 to have a previous season"
    prev_season = season - 1
    prev_home_league_info = get_league_info_of_team(home_team_id, prev_season)
    prev_away_league_info = get_league_info_of_team(away_team_id, prev_season)
    row['home_prev_level_diff'] = league_level - prev_home_league_info[1]
    row['away_prev_level_diff'] = league_level - prev_away_league_info[1]
    home_prev_league_id = prev_home_league_info[0]
    away_prev_league_id = prev_away_league_info[0]
    # add the number of wins after last round for both teams in the previous league
    max_season_games = 22
    if prev_season == get_current_season() - 1 and prev_home_league_info[1] <= 3:
        row['home_prev_season_wins'] = converter.get_team_wins_before_round(home_team_id, prev_season, home_prev_league_id, max_season_games+1)
    if prev_season == get_current_season() - 1 and prev_away_league_info[1] <= 3:
        row['away_prev_season_wins'] = converter.get_team_wins_before_round(away_team_id, prev_season, away_prev_league_id, max_season_games+1)
    valid_league_games.append(row)

# Convert to DataFrame
valid_league_games = pd.DataFrame(valid_league_games)

In [ ]:
valid_league_games.columns

In [ ]:
# X columns for demand prediction
columns_to_keep = [
    'game_id',
    'league_level',
    'home_wins',
    'home_points_diff',
    'away_wins',
    'away_points_diff',
    'round_number',
    'home_prev_level_diff',
    'away_prev_level_diff',
    'home_prev_season_wins',
    'away_prev_season_wins',
    'home_won_prev_game',
    'away_won_prev_game',
    'bleachers_price',
    'lower_tier_price',
    'courtside_price',
    'luxury_boxes_price',
    'is_tv_game',
]

In [ ]:
X = valid_league_games[columns_to_keep]
X.sort_values(by='home_wins', ascending=False)

In [ ]:
target_columns = [
    'bleachers_attendance',
    'lower_tier_attendance',
    'courtside_attendance',
    'luxury_boxes_attendance',
]

In [ ]:
y = valid_league_games[target_columns]
y.sort_values(by='bleachers_attendance', ascending=False)

In [ ]:
# store X and y
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
X.to_csv('X_league.rs_games.csv', index=False)
y.to_csv('y_league.rs_games.csv', index=False)